# PARTE 1: Algoritmo Naïve Bayes

Nesta primeira parte do Trabalho você irá aplicar o algoritmo de Naïve Bayes na base de dados de risco de crédito discutida em aula. Para isso você deve primeiramente importar as bibliotecas necessárias.

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [23]:
# importe a base de dados de risco de crédito e nomeie com: dataset_risco_credito_risco_credito
dataset_risco_credito_risco_credito = pd.read_csv('dataset_risco_credito.csv')

In [24]:
# imprima a base de dados para uma primeira avaliação dos dados
# Mapear os valores da coluna 'renda' para rótulos legíveis (ex: 'Renda de 0 a 15')
renda_map = {
    '0_15': 'Renda de 0 a 15',
    '15_35': 'Renda de 15 a 35',
    'acima_35': 'Renda acima de 35'
}
ds = dataset_risco_credito_risco_credito

# criar coluna com rótulos legíveis; se algum valor não tiver mapeamento, mantém o original
ds['renda_label'] = ds['renda'].map(renda_map).fillna(ds['renda'])

fig = px.bar(
    ds,
    x='renda_label',            
    color='risco',              
    barmode='group',           
    color_discrete_map={'alto':'#EF553B', 'moderado':'#FFA15A', 'baixo':'#00CC96'}, # cores intuitivas
    labels={'renda_label':'Faixa de Renda', 'count':'Quantidade de Registros', 'risco':'Risco de Crédito'},
    title='Distribuição de Risco de Crédito por Faixa de Renda'
)


fig.update_layout(
    title_font_size=22,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=16,
    plot_bgcolor='#F9F9F9'
)

fig.show()

# 1 - Pré-Processamento dos dados

a) DIVISÃO DA BASE DE DADOS

Separe a base de dados dataset_risco_credito_risco_credito em:
 - variável x, com nome: X_risco_credito
 - classe y, com nome: y_risco_credito

DICA: você pode utilizar .iloc para selecionar as colunas da matriz e .values para converter para o numpy array.

In [25]:
X_risco_credito = dataset_risco_credito_risco_credito.iloc[:, 0:-2].values   # todas as colunas, exceto a última
y_risco_credito = dataset_risco_credito_risco_credito.iloc[:, -2].values     # apenas a última coluna


print("X_risco_credito:\n", X_risco_credito[:5])  # mostra as 5 primeiras linhas
print("\ny_risco_credito:\n", y_risco_credito[:5])

X_risco_credito:
 [['ruim' 'alta' 'nenhuma' '0_15']
 ['desconhecida' 'alta' 'nenhuma' '15_35']
 ['desconhecida' 'baixa' 'nenhuma' '15_35']
 ['desconhecida' 'baixa' 'nenhuma' 'acima_35']
 ['desconhecida' 'baixa' 'nenhuma' 'acima_35']]

y_risco_credito:
 ['alto' 'alto' 'moderado' 'alto' 'baixo']


b) APLICAR LABEL ENCODER

Perceba que seus dados possuem atributos categóricos (string). Porém, para aplicar esses dados em um algoritmo de aprendizado você precisa transformá-lo em atributo numérico. 

Como você pode resolver isso?

DICA: Veja o que é e como aplicar o Label Enconder em: https://youtu.be/nLKEkBAbpQo 

In [ ]:
# Transformar todos os parâmetros categóricos em valores numéricos usando um LabelEncoder por coluna
from sklearn.preprocessing import LabelEncoder
# Criar um encoder por atributo (coluna) e armazená-los para uso posterior em novos casos
encoders = []
for i in range(X_risco_credito.shape[1]):
    le = LabelEncoder()
    X_risco_credito[:, i] = le.fit_transform(X_risco_credito[:, i])
    encoders.append(le)
# Criar um encoder separado para a variável alvo (y) para permitir inverse_transform das previsões
y_encoder = LabelEncoder()
y_risco_credito = y_encoder.fit_transform(y_risco_credito)

print("X_risco_credito (numérico):\n", X_risco_credito[:5])
print("\ny_risco_credito (numérico):\n", y_risco_credito[:5])

X_risco_credito (numérico):
 [[2 0 1 0]
 [1 0 1 1]
 [1 1 1 1]
 [1 1 1 2]
 [1 1 1 2]]

y_risco_credito (numérico):
 [0 0 2 0 1]


c) SALVAR O ARQUIVO PRÉ-PROCESSADO

In [27]:
# como salvar o arquivo:
import pickle
with open('risco_credito.pkl', 'wb') as f:
  pickle.dump([X_risco_credito, y_risco_credito], f)

# 2 - Algoritmo Naïve Bayes

In [28]:
# importar da biblioteca sklearn o pacote Nayve Bayes
# utilizamos a distribuição estatística Gaussiana (classe GaussianNB) ou distribuição normal pois é mais usado para problemas genéricos
from sklearn.naive_bayes import GaussianNB

In [29]:
# Criar o objeto Nayve Bayes
naiveb_risco_credito = GaussianNB()

a) TREINAR O ALGORITMO

Para treinar o algoritmo, você deve gerar a tabela de probabilidades. Para isso, você pode utilizar **.fit** para gerar a tabela.

DICA: O 1º parametro são os atributos/características (x) e o 2º parametro é a classe (y).

OBS: Não se preocupe, o algoritmo faz a correção laplaciana automaticamente :) .

In [30]:
# Treinar o modelo Naive Bayes usando os dados pré-processados
naiveb_risco_credito.fit(X_risco_credito, y_risco_credito)

,priors,None
,var_smoothing,1e-09


b) FAZER A PREVISÃO

Utilize **.predict** para fazer a previsão realizada no exemplo em sala.

i) história boa, dívida alta, garantia nenhuma, renda > 35

ii) história ruim, dívida alta, garantia adequada, renda < 15

Verifique nos slides se seu resultado está correto!

In [ ]:
# Mostrar a ordem das features originais
colunas = dataset_risco_credito_risco_credito.columns[:-1]  # todas exceto a última (risco)
print("Features do modelo (ordem):", list(colunas))
print("\nClasses do modelo (números):", naiveb_risco_credito.classes_)
# Usar o encoder da variável alvo para transformar as classes numéricas em rótulos legíveis
print("Classes do modelo (rótulos):", y_encoder.inverse_transform(naiveb_risco_credito.classes_))

Features do modelo (ordem): ['historia', 'divida', 'garantias', 'renda', 'risco']

Classes do modelo (números): [0 1 2]
Classes do modelo (rótulos): ['alto' 'baixo' 'moderado']


In [ ]:
# Preparar os novos casos para previsão - seguindo a ordem exata das features do dataset
novos_creditos = [
    # historia, divida, garantias, renda
    ['boa', 'alta', 'nenhuma', 'acima_35'],    # Caso i
    ['ruim', 'alta', 'adequada', '0_15']       # Caso ii
]

# Transformar os novos casos usando os mesmos encoders (um por coluna) criados no pré-processamento
novos_creditos_encoded = []
for caso in novos_creditos:
    caso_encoded = []
    # zip garante que cada valor seja transformado com o encoder correspondente à sua coluna
    for valor, encoder in zip(caso, encoders):
        caso_encoded.append(encoder.transform([valor])[0])
    novos_creditos_encoded.append(caso_encoded)

# Converter para array numpy para garantir formato correto
novos_creditos_encoded = np.array(novos_creditos_encoded)

# Fazer as previsões (retorna rótulos numéricos)
previsoes = naiveb_risco_credito.predict(novos_creditos_encoded)

# Converter as previsões de volta para rótulos legíveis usando o encoder da variável alvo
previsoes_rotulos = y_encoder.inverse_transform(previsoes)

# Mostrar as previsões de forma mais organizada
print("\nPrevisões:")
for i, (caso, previsao) in enumerate(zip(novos_creditos, previsoes_rotulos), 1):
    print(f"\nCaso {i}:")
    print(f"História: {caso[0]}")
    print(f"Dívida: {caso[1]}")
    print(f"Garantias: {caso[2]}")
    print(f"Renda: {caso[3]}")
    print(f"\nPrevisão de risco: {previsao}")

NameError: name 'encoders' is not defined